# Training and validating on simulated data

In this notebook we go through the construction of a training and a testing dataset as has been done in the article.

We then train our model and finally validate it with the testing dataset.



In [1]:
import numpy as np
import mccd.auxiliary_fun as mccd_aux
import mccd
from astropy.io import fits

import random

import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


In [2]:
plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 20}
mpl.rc('font', **font)

plot_style = {
                'figure.figsize': (6,6),
                'figure.dpi': 100,
                'figure.autolayout':True,
                'lines.linewidth': 4,
                'lines.linestyle': '-',
                'lines.marker': 'o',
                'lines.markersize': 10,
                'legend.fontsize': 20,
                'legend.loc': 'best',
                'axes.titlesize': 24,
                'axes.grid': False,
                'axes.grid.which': 'major',
                'axes.grid.axis': 'both'}
mpl.rcParams.update(plot_style)

def plot_fun(star_img, cmap='gist_stern'):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    im1 = ax1.imshow(star_img, interpolation='None',cmap=cmap)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(im1, cax=cax, extend='both')
    cbar.minorticks_on()
    fig.tight_layout()
    ax1.set_xticks([]);ax1.set_yticks([])
    ax1.axis('off')
    plt.show()

loc2glob = mccd.mccd_utils.Loc2Glob()

# Pre-defined colormap
top = mpl.cm.get_cmap('Oranges_r', 128)
bottom = mpl.cm.get_cmap('Blues', 128)
newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')

def plot_ccds_fun(positions,mom,title):
    fig = plt.figure(figsize=(8, 8))
    ax1 = fig.add_subplot(111)
    for ccd_it in range(40):
        xs,ys = loc2glob.shift_coord(ccd_it)
        rect = mpl.patches.Rectangle((xs,ys),2048,4612,linewidth=1,edgecolor='r',facecolor='none')
        ax1.add_patch(rect)

    im1 = ax1.scatter(positions[:, 0], positions[:, 1], s=5,c=mom, marker='*', cmap=newcmp)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax1.set_xticks([]);ax1.set_yticks([])
    ax1.axis('off')
    ax1.set_title(title)
    plt.show()
    

# Generate training and testing datasets

In [4]:
input_pos_path = './../data/sim_inputs/train_positions.npy'
input_ccd_path = './../data/sim_inputs/train_ccd_list.npy'
output_path = './../data/mccd_inputs/'

sim_dataset_generator = mccd_aux.GenerateSimDataset(input_pos_path, input_ccd_path, output_path)

sim_dataset_generator.load_data()
# Posibility to define the simulation parameters
sim_dataset_generator.generate_train_data()  # Using default parameters
# Defining the testing grid
sim_dataset_generator.generate_test_data(x_grid=5, y_grid=10)


KeyboardInterrupt: 

In [3]:
input_pos_path = './../data/sim_inputs/train_positions.npy'
input_ccd_path = './../data/sim_inputs/train_ccd_list.npy'
output_path = './../data/mccd_inputs/'

# Get the dataset paths
ext = '.fits'
catalog_id = 1111111

train_cat_path = output_path + 'train_star_selection-' + str(catalog_id) + ext
test_cat_path = output_path + 'test_star_selection-' + str(catalog_id) + ext


# Check the trainiung dataset

In [4]:
train_cat = fits.open(train_cat_path)[1]

In [5]:
pos = train_cat.data['GLOB_POSITION_IMG_LIST']
#plot_ccds_fun(pos,train_cat.data['TRUE_E1_HSM'],'e1 star positions')
#plot_ccds_fun(pos,train_cat.data['TRUE_E2_HSM'],'e2 star positions')
#plot_ccds_fun(pos,2.*train_cat.data['TRUE_SIG_HSM']**2,'R2 star positions')



In [6]:
# Load the data
train_cat = fits.open(train_cat_path)

# Prepare for the training

We need to prepare the elements we will feed the MCCD algorithm.

- Each input should be a list of elements.
- Each position in the list will correspond to the data from one CCD.
- The CCD id of each position should be included in the ``ccd_list``.


- Data in fits file are concatenated over the first dimension into a large np.ndarray.
- We need to split the data to have it assigned for the corresponding CCD.



In [7]:
train_cat[1].columns

ColDefs(
    name = 'VIGNET_LIST'; format = '2601D'; dim = '(51, 51)'
    name = 'GLOB_POSITION_IMG_LIST'; format = '2D'; dim = '(2,)'
    name = 'MASK_LIST'; format = '2601D'; dim = '(51, 51)'
    name = 'CCD_ID_LIST'; format = '1D'
    name = 'TRUE_E1_HSM'; format = '1D'
    name = 'TRUE_E2_HSM'; format = '1D'
    name = 'TRUE_SIG_HSM'; format = '1D'
)

In [8]:
# Extract the data from the fits catalog 
positions = np.copy(train_cat[1].data['GLOB_POSITION_IMG_LIST'])
stars = np.copy(train_cat[1].data['VIGNET_LIST'])
ccds = np.copy(train_cat[1].data['CCD_ID_LIST']).astype(int)
ccds_unique = np.unique(np.copy(train_cat[1].data['CCD_ID_LIST'])).astype(int)
# Generate the masks
masks = mccd.utils.handle_SExtractor_mask(stars,thresh=-1e5)

# Generate the list format needed by the MCCD package
pos_list = [positions[ccds == ccd] for ccd in ccds_unique]
star_list = [mccd.utils.rca_format(stars[ccds == ccd]) for ccd in ccds_unique]
mask_list = [mccd.utils.rca_format(masks[ccds == ccd]) for ccd in ccds_unique]
ccd_list = [ccds[ccds == ccd].astype(int) for ccd in ccds_unique]
ccd_list = [np.unique(_list)[0].astype(int) for _list in ccd_list]
SNR_weight_list = None  # We wont use any weighting technique as the SNR is constant over the Field of View


# MCCD train parameters

In [9]:
# Parameters

# MCCD instance
n_comp_loc = 4
d_comp_glob = 6
filters = None
ksig_loc = 0
ksig_glob = 0

# MCCD fit
psf_size = 6.2
psf_size_type = 'R2'
n_eigenvects = 5
n_iter_rca = 1
nb_iter_glob = 2 
nb_iter_loc = 2
nb_subiter_S_loc = 300
nb_subiter_A_loc = 500
nb_subiter_S_glob = 100
nb_subiter_A_glob = 200
loc_model = 'hybrid'


# Build the paramter dictionaries
mccd_inst_kw = {'n_comp_loc': n_comp_loc, 'd_comp_glob': d_comp_glob,
                'filters': filters,       'ksig_loc': ksig_loc,
                'ksig_glob':ksig_glob}

mccd_fit_kw = {'psf_size': psf_size,                  'psf_size_type':psf_size_type,
              'n_eigenvects': n_eigenvects,          'nb_iter':n_iter_rca,
              'nb_iter_glob':nb_iter_glob,           'nb_iter_loc':nb_iter_loc,
              'nb_subiter_S_loc':nb_subiter_S_loc,   'nb_subiter_A_loc':nb_subiter_A_loc,
              'nb_subiter_S_glob':nb_subiter_S_glob, 'nb_subiter_A_glob':nb_subiter_A_glob,
              'loc_model':loc_model}


# Launch the training

- Then save the fitted model

In [10]:
# Instanciate the class
mccd_instance = mccd.MCCD(**mccd_inst_kw, verbose=True)
# Launch the training
S, A_loc, A_glob, alpha, pi = mccd_instance.fit(star_list, pos_list, ccd_list, mask_list,
                                                SNR_weight_list, **mccd_fit_kw)

fitted_model_path = output_path + '/fitted_model' + str(catalog_id)
mccd_instance.quicksave(fitted_model_path)


Running basic initialization tasks...
... Done.
Constructing local spatial constraint...


/Users/oa265351/opt/miniconda3/envs/denoising/lib/python3.9/site-packages/mccd-0.0.4-py3.9.egg/mccd/utils.py:523: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  self.VT = np.vstack((eigenvect for eigenvect in list_eigenvects))


... Done.
Building position polynomials...
... Done.


/Users/oa265351/opt/miniconda3/envs/denoising/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


1/1 [==============================] - 5s 5s/step - loss: 0.1132 - val_loss: 0.1265
allo
(51, 51, 10)
 - Converged!


...

100% (300 of 300) |######################| Elapsed Time: 0:00:03 Time:  0:00:03
  2% (6 of 300) |                        | Elapsed Time: 0:00:02 ETA:   0:02:15

 - Converged!


100% (300 of 300) |######################| Elapsed Time: 0:00:03 Time:  0:00:03
  2% (6 of 300) |                        | Elapsed Time: 0:00:02 ETA:   0:02:24

 - Converged!


100% (300 of 300) |######################| Elapsed Time: 0:00:03 Time:  0:00:03


# Validation

In [14]:
# Import
test_cat = fits.open(test_cat_path)[1]
val_dict = mccd_aux.mccd_validation(fitted_model_path, test_cat)


In [15]:
val_dict.keys()

dict_keys(['PSF_VIGNET_LIST', 'PSF_MOM_LIST', 'STAR_MOM_LIST', 'GLOB_POSITION_IMG_LIST', 'VIGNET_LIST', 'MASK_LIST', 'CCD_ID_LIST'])

In [16]:
test_stars = val_dict['VIGNET_LIST']
matched_PSFs = val_dict['PSF_VIGNET_LIST']

e1_psf = val_dict['PSF_MOM_LIST'][:,0]
e2_psf = val_dict['PSF_MOM_LIST'][:,1]
R2_psf = 2.*val_dict['PSF_MOM_LIST'][:,2]**2

e1_star = val_dict['STAR_MOM_LIST'][:,0]
e2_star = val_dict['STAR_MOM_LIST'][:,1]
R2_star = 2.*val_dict['STAR_MOM_LIST'][:,2]**2


In [17]:
pix_RMSE = np.sqrt(np.mean((test_stars - matched_PSFs)**2))
e1_RMSE = np.sqrt(np.mean((e1_star - e1_psf)**2))
e2_RMSE = np.sqrt(np.mean((e2_star - e2_psf)**2))
R2_RMSE = np.sqrt(np.mean(((R2_star - R2_psf)/R2_star)**2))

print('Pixel RMSE = %.5e'%(pix_RMSE))
print('e1 RMSE = %.5e'%(e1_RMSE))
print('e2 RMSE = %.5e'%(e2_RMSE))
print('R2 RMSE = %.5e'%(R2_RMSE))


Pixel RMSE = 2.64408e-04
e1 RMSE = 2.38796e-02
e2 RMSE = 2.75909e-02
R2 RMSE = 3.97102e-02


## Plot a random test star and its model reconstruction

In [18]:
idx = random.randint(0,test_stars.shape[0])

plot_fun(test_stars[idx,:,:])
plot_fun(matched_PSFs[idx,:,:])
plot_fun(test_stars[idx,:,:] - matched_PSFs[idx,:,:])


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 600x600 with 2 Axes>